In [2]:
from commons.db_manager import DbManager
import numpy as np
import pandas as pd
from  datetime import date, timedelta
import plotly.express as px

In [3]:
db = DbManager("root", "bandtec123", "localhost", "processamento_db")
    # api = ApiClient("http://localhost")

data_frame = db.read("SELECT * FROM acessos").astype({
        'fimAcesso':'datetime64[ns]', 
        'inicioAcesso':'datetime64[ns]', 
        })
data_frame

Conectado ao banco


,idAcessos,idConsumidor,inicioAcesso,fimAcesso,fkEcommerce
0,1,1,2021-09-18 10:30:00,2021-09-18 12:35:31,1
1,2,2,2021-09-18 09:15:15,2021-09-18 12:35:51,2
2,3,2,2021-09-18 10:30:12,2021-09-18 12:35:52,2
3,4,4,2021-09-18 10:30:55,2021-09-18 12:35:53,3


# Obtendo Duração da Sessão

In [17]:
data_frame['duracao_sessao'] =  (data_frame.fimAcesso - data_frame['inicioAcesso']).dt.seconds
data_frame

,fkEcommerce,idConsumidor,quantidade,total_Duracao_sessao
0,1,1,1,7531
1,2,2,2,19576
2,3,4,1,7498


# Número de acessos por ecommerce

In [15]:
data_frame.groupby('fkEcommerce').agg(quantidade=('idAcessos','count')).reset_index()

,fkEcommerce,quantidade
0,1,1
1,2,2
2,3,1


# Número de acessos por cliente

In [6]:
data_frame.groupby('idConsumidor').agg({'idAcessos':'count'}).reset_index()

,idConsumidor,idAcessos
0,1,1
1,2,2
2,4,1


# Número de acessos mensais por ecommerce

In [7]:
month_dataframe_ecommerce = data_frame.copy()
month_dataframe_ecommerce['mes_de_acesso'] = data_frame['inicioAcesso'].dt.month
month_dataframe_ecommerce.groupby(['mes_de_acesso', 'fkEcommerce']).agg({'idAcessos':'count'})

idAcessos
mes_de_acesso fkEcommerce           
9             1                    1
              2                    2
              3                    1

# Número de acessos mensais por cliente

In [8]:
month_dataframe = data_frame.copy()
month_dataframe['mes_de_acesso'] = data_frame['inicioAcesso'].dt.month
month_dataframe.groupby(['mes_de_acesso', 'idConsumidor']).agg({'idAcessos':'count'})

idAcessos
mes_de_acesso idConsumidor           
9             1                     1
              2                     2
              4                     1

# Duração de acesso por ecommerce (Em horas)

In [9]:
data_frame_sum_ecommerce = data_frame.groupby('fkEcommerce')['duracao_sessao'].sum().reset_index()
data_frame_sum_ecommerce['horas_totais'] = data_frame_sum_ecommerce['duracao_sessao'] / 60 / 60
data_frame_sum_ecommerce

,fkEcommerce,duracao_sessao,horas_totais
0,1,7531,2.091944
1,2,19576,5.437778
2,3,7498,2.082778


# Duração de acesso por clientes (Em horas)

In [10]:
data_frame_sum = data_frame.groupby('idConsumidor')['duracao_sessao'].sum().reset_index()
data_frame_sum['horas_totais'] = data_frame_sum['duracao_sessao'] / 60 / 60
data_frame_sum

,idConsumidor,duracao_sessao,horas_totais
0,1,7531,2.091944
1,2,19576,5.437778
2,4,7498,2.082778
